In [1]:
from legacy_static import *
import pandas as pd
import scipy.io
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
import random
from canny_edge_detector_pytorch import *
from torchvision.transforms import ToPILImage

C:\Users\Sai\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from model_architectures import *
def convert_edge_v3(fi,save, factor=2.5):
    # 0: Import Model
    device = "cuda"
    model = CNN(input_shape=(1, 10, 10), num_classes=96).to(device)
    model.load_state_dict(torch.load('artifacts/cnn_ascii_classifier.pth'))
    model.eval()
    # 1: detect edges using Stefan Pitur's implementation
    low_thresh = 10
    high_thresh = 60
    og_img = cv2.imread(fi)
    tensor = canny_edge_detection([og_img],low_threshold=low_thresh, high_threshold=high_thresh)[0]
    tensor = tensor * 255.0
    tensor = tensor.byte()
    to_pil = ToPILImage()
    pil_img = to_pil(tensor).convert("L")

    # 2: Resize
    height, width = og_img.shape[:2]
    scale = width / (factor*500)
    width, height = width / scale, height / (scale*2)
    width, height = int(width), int(height)

    # final = ImageOps.invert(pil_img)
    final = pil_img
    final = final.resize((width,height))

    # original_img = original_img.resize((width,height))
    # final.show()      
    
    # 3: Convert to 0s and 1s for bnw
    img_array_edges = np.array(final)
    # original_img = np.array(original_img)
    f_vec = np.vectorize(f4)
    img_array_edges = f_vec(img_array_edges)
    fin = ""

    # SAVING TRAIN DATA
    data = []
    labels = []
    counter = 0

    # 4: Take 10x10 tiles and replace with ASCII character
    inc = 10
    for li in range(0,math.floor(len(img_array_edges) / inc) * inc,inc): #Floor function to use only even bounds. 
    #     Odd bounds have data loss on last col/row
        for i in range(0,math.floor(len(img_array_edges[li]) / inc) * inc,inc):
            # Subsetting the tile itself
            subs = []
            subs.append([xi[i:i+inc] for xi in img_array_edges[li:li+inc]])
            subs = np.array(subs)

            subs_og = []
            subs_og.append([xi[i:i+inc] for xi in img_array_edges[li:li+inc]])
            subs_og = np.array(subs_og)
            # checking the minimum distance between each character vector and tile vector
            min = 0
            min_dist = np.linalg.norm(edge_vectors3[0]-subs.flatten())
            if not (np.all(subs == 0)):
                for j in range(0,len(edge_vectors3)):
                    new_mind_dist = np.linalg.norm(edge_vectors3[j]-subs.flatten())

                    if (new_mind_dist<min_dist):
                        min = j
                        min_dist = new_mind_dist
            
            # SAVING TRAIN DATA
            # if counter < 100000
            data.append(subs[0])
            labels.append(min)

            

            fin = fin + chr(min+32)
        fin = fin + '\n'

    # Normalize data
    data = np.array(data)
    labels = np.array(labels)
    data = data / 255.0
    data = data.reshape((len(data), 10, 10, 1))  # (num_samples, height, width, channels)
    flattened_images = data.reshape(data.shape[0], -1)
    reshaped_data = np.concatenate((labels.reshape(-1, 1).astype(np.int8), flattened_images.astype(np.float64)), axis=1)

    df = pd.DataFrame(reshaped_data)

    df = df.reset_index(drop=True)
    df.to_csv('archive/ascii_character_classification_sanrio_data_v2.csv', mode='a', header=None, index=False)

    
    # 5: Output to terminal
    # print(fin)
    # 6: Save to output file
    # if(save):
    fi2 = os.path.splitext(fi)[0].replace('images', 'text') + ".txt"
    with open(fi2,'w') as f:
        f.write(fin)



def convert_edge_cnn(fi,save, factor=2.5):
    # 0: Import Model
    device = "cuda"
    model = CNN(input_shape=(1, 10, 10), num_classes=96).to(device)
    model.load_state_dict(torch.load('artifacts/cnn_ascii_classifier.pth'))
    model.eval()
    # 1: detect edges using Stefan Pitur's implementation
    low_thresh = 10
    high_thresh = 60
    og_img = cv2.imread(fi)
    tensor = canny_edge_detection([og_img],low_threshold=low_thresh, high_threshold=high_thresh)[0]
    tensor = tensor * 255.0
    tensor = tensor.byte()
    to_pil = ToPILImage()
    pil_img = to_pil(tensor).convert("L")

    # 2: Resize
    height, width = og_img.shape[:2]
    scale = width / (factor*500)
    width, height = width / scale, height / (scale*2)
    width, height = int(width), int(height)

    # final = ImageOps.invert(pil_img)
    final = pil_img
    final = final.resize((width,height))

    # original_img = original_img.resize((width,height))
    # final.show()      
    
    # 3: Convert to 0s and 1s for bnw
    img_array_edges = np.array(final)
    # original_img = np.array(original_img)
    f_vec = np.vectorize(f4)
    img_array_edges = f_vec(img_array_edges)
    fin = ""

    # SAVING TRAIN DATA
    data = []
    labels = []
    counter = 0

    # 4: Take 10x10 tiles and replace with ASCII character
    inc = 10
    tiles = []
    for li in range(0, math.floor(len(img_array_edges) / inc) * inc, inc):
        for i in range(0, math.floor(len(img_array_edges[li]) / inc) * inc, inc):
            subs = img_array_edges[li:li + inc, i:i + inc]
            # subs = subs / 255  # Normalize
            tiles.append(subs)
        
    tiles = np.array(tiles)
    tiles_tensor = torch.tensor(tiles).unsqueeze(1).to(torch.float32).to(device)

    # 5: Perform batch prediction
    with torch.no_grad():
        outputs, _ = model(tiles_tensor)
        probabilities = F.softmax(outputs, dim=1)
        _, indices = torch.max(probabilities, 1)
    
    # 6: Convert predictions to ASCII characters and build the output string
    fin = ""
    index = 0
    for li in range(0, math.floor(len(img_array_edges) / inc) * inc, inc):
        for i in range(0, math.floor(len(img_array_edges[li]) / inc) * inc, inc):
            char = chr(indices[index].item() + 32)
            fin += char
            index += 1
        fin += '\n'

    
    # 5: Output to terminal
    # print(fin)
    # 6: Save to output file
    # if(save):
    fi2 = os.path.splitext(fi)[0].replace('images', 'text') + ".txt"
    with open(fi2,'w') as f:
        f.write(fin)

Using device: cuda


In [3]:
import os
from PIL import Image, ImageEnhance

# Function to enhance the image
def enhance_image(image):
    # Enhance contrast
    enhancer = ImageEnhance.Contrast(image)
    image = enhancer.enhance(1.5)  # Increase contrast by 50%
    
    # Enhance brightness
    enhancer = ImageEnhance.Brightness(image)
    image = enhancer.enhance(1.2)  # Increase brightness by 20%
    
    # Enhance sharpness
    enhancer = ImageEnhance.Sharpness(image)
    image = enhancer.enhance(2.0)  # Increase sharpness by 100%
    
    return image

# Function to upscale images
def upscale_image(input_path, output_path, size=(1000, 1000)):
    with Image.open(input_path) as img:
        # Check if the image is 150x150 pixels
        # if img.size == (150, 150):
        # Enhance the image
        if img.mode == 'RGB':
            img = enhance_image(img)
        
        # Upscale the image
        img = img.resize(size, Image.ANTIALIAS)
        
        # Save the image to the output path
        img.save(output_path)
        print(f"Upscaled and saved image: {output_path}")

# Function to traverse directories and find images
def upscale_images_in_directory(directory):
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif')):
                input_path = os.path.join(root, file)
                output_path = input_path  # Overwrite the original image
                try:
                    upscale_image(input_path, output_path)
                except Exception as e:
                    print(f"Error processing {input_path}: {e}")

# Specify the root directory to start
root_directory = 'Logos/images'

# Start the upscaling process
# upscale_images_in_directory(root_directory)


In [6]:
# path = "C:\\Users\\Sai\\Work\\Transformer\\data"
#  Source: Google Quickdraw
# path = "anime_pics"
# Source: https://www.kaggle.com/datasets/prasoonkottarathil/gananime-lite
# path = "Logos/images"
# Source: https://www.kaggle.com/datasets/kkhandekar/popular-brand-logos-image-dataset?select=Logos
path = "Sanrio/images"
# Source: https://www.kaggle.com/datasets/vnxiclaire/sanrio-character-image-dataset/data
# path = "data/images"
for f in os.listdir(path):
    print(f)
    # random_number = random.uniform(1, 3)
    convert_edge_v3(os.path.join(path,f), save=False, factor=3)

if not os.path.exists(path):
    print(f"Path '{path}' does not exist.")
    
# else:
#     counter = 0
#     for f in os.listdir(path):
#         counter+=1
#         if counter > 0:
#             print(f"Files Processed: {counter}")
#             file_path = os.path.join(path, f)
#             if os.path.isfile(file_path):
#                 cv2_img = DetectEdges(file_path)
#                 if cv2_img.dtype is not None:
#                     pil_img = Image.fromarray(cv2_img).convert("L")
#                     pil_img.save(os.path.join("EdgeMaps", f))
#                 else:
#                     print(f"Error processing '{file_path}'.")
#             else:
#                 print(f"'{file_path}' is not a file.")


02_nobunyan_a.png
03_yukinyan_a.png
04_datenyan_a.png
05_nyanbee_a.png
06_keijironyan_a.png
07_kenshinnyan_a.png
08_shingennyan_a.png
09_imanyan_a.png
10_nyanbu_a.png
11_jibunyan_a.png
12_nyanrin_a.png
13_nyanba_a.png
14_kiyomanya_a.png
15_chousokabenyan_a.png
16_iepon_a.png
accyangaichiban_b.png
aggressiveretsuko_b.png
aggressiveretsuko_sub_02.png
aggressiveretsuko_sub_03.png
aggressiveretsuko_sub_04.png
aggressiveretsuko_sub_05.png
aggressiveretsuko_sub_06.png
aggressiveretsuko_sub_07.png
aggressiveretsuko_sub_08.png
aggressiveretsuko_sub_09.png
aggressiveretsuko_sub_10.png
aggressiveretsuko_sub_11.png
aggressiveretsuko_sub_12.png
aggressiveretsuko_sub_13.png
aggressiveretsuko_sub_14.png
aggressiveretsuko_sub_15.png
aggressiveretsuko_sub_16.png
aggressiveretsuko_sub_17.png
aggressiveretsuko_sub_18.png
aggressiveretsuko_sub_19.png
aggressiveretsuko_sub_20.png
aggressiveretsuko_sub_21.png
aggressiveretsuko_sub_22.png
aggressiveretsuko_sub_23.png
aggressiveretsuko_sub_24.png
aggressiver

In [5]:
df = pd.read_csv("archive/ascii_character_classification_logos_data.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'archive/ascii_character_classification_logos_data.csv'

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel91,pixel92,pixel93,pixel94,pixel95,pixel96,pixel97,pixel98,pixel99,pixel100
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
import os
import re

# Function to replace the first character of each line with a space
def replace_first_char(file_path):
    # Read the original file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Replace the first character of each line with a space
    modified_lines = [' ' + line[1:] if line else line for line in lines]

    # Write the modified lines to a new file
    with open(file_path, 'w') as file:
        file.writelines(modified_lines)

# Replace the first character in 'example.txt'

# Specify the folder path
folder_path = 'Logos/text'

counter = 0
# Iterate over each file in the folder
for filename in os.listdir(folder_path):
    counter += 1
    if filename.endswith('.txt'):
        file_path = os.path.join(folder_path, filename)
        
        # Replace characters adjacent to \n with a space
        replace_first_char(file_path)
        
        

        print(f'Processed {filename}')


Processed 2016-super-bowl-vector-logo-400x400.txt
Processed a-stars-alpinestars-vector-logo-400x400.txt
Processed aareal-bank-vector-logo-400x400.txt
Processed abercrombie-and-fitch-black-vector-logo-400x400.txt
Processed abercrombie-fitch-a-f-vector-logo-400x400.txt
Processed abs-wheels-vector-logo-400x400.txt
Processed abta-logo-vector-download-400x400.txt
Processed abu-dhabi-logo.txt
Processed ac-cuneo-1905-vector-logo-400x400.txt
Processed ac-dc-black-vector-logo-400x400.txt
Processed ac-milan-eps-vector-logo-400x400.txt
Processed ac-pisa-1909-vector-logo-400x400.txt
Processed accept-logo-vector-download.txt
Processed ace-cafe-london-logo-vector-download.txt
Processed adidas-3-vector-logo-400x400.txt
Processed adidas-black-vector-logo-400x400.txt
Processed adidas-celebrate-originality-vector-logo-400x400.txt
Processed Adidas-logo-400x400.txt
Processed adidas-originals-logo-400x400.txt
Processed Adidas-Trefoil-logo-vector-for-free-download.txt
Processed Adidas-Y-3-logo-vector-free-d